<h1>Содержание<span class="tocSkip"></span></h1>
</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Подготовим-данные-к-подаче-в-модели" data-toc-modified-id="Подготовим-данные-к-подаче-в-модели-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовим данные к подаче в модели</a></span></li><li><span><a href="#Обучим-3-модели" data-toc-modified-id="Обучим-3-модели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучим 3 модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [31]:
import pandas as pd
from pymystem3 import Mystem
import re
import nltk

from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, GridSearchCV 

from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import f1_score

from tqdm import notebook, tqdm, tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Подготовка данных

Рассмотрим данные

In [14]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [15]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [17]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [18]:
data = data.drop(['Unnamed: 0'], axis=1)

In [19]:
data = data.reset_index()

Изначально у нас было 3 столбца: Unnamed: 0, text, toxic. 1 убрали, так как он не имел никакого смысла и дублировал индексы. 
<br/>Второй столбец содержит строи с текстам, которые по итогу будут признаками в моделях. 
<br/>3 столбец - целевой признак: значение "1" текст токсичный, "0" - нет. 

Очистим текст от неожиданных символов, учитывая регистр, и лемматизизуем его.

In [20]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    return ' '.join(text.split())

In [21]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [22]:
def wnl_lemmatize(text):
    wnl = WordNetLemmatizer()
    s = []
    for word in text.split():
        s.append(wnl.lemmatize(word, get_wordnet_pos(word)))
    return ' '.join(s)

In [23]:
notebook.tqdm.pandas()
data['clear'] = data.text.progress_apply(clear_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [24]:
notebook.tqdm.pandas()
data['lem_text'] = data.clear.progress_apply(wnl_lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [25]:
data = data.drop(['index', 'clear', 'text'], axis=1)

In [26]:
data.head(20)

,toxic,lem_text
0,0,explanation why the edits make under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not try to edit war it s ju...
3,0,more i can t make any real suggestion on impro...
4,0,you sir be my hero any chance you remember wha...
5,0,congratulation from me a well use the tool wel...
6,1,cocksucker before you piss around on my work
7,0,your vandalism to the matt shirvington article...
8,0,sorry if the word nonsense be offensive to you...
9,0,alignment on this subject and which be contrar...


Наш датасет готов к подаче в модели, отсалось его разделить на тренировочную и тестовую выборки, выделить целевой признак и и векторизировать признаки.

## Обучение

### Подготовим данные к подаче в модели

In [33]:
features = data.drop(columns=['toxic'])
target = data['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            shuffle=True, 
                                                                            test_size=0.2, random_state=12345)

In [34]:
features_train.shape

(127433, 1)

In [35]:
features_test.shape

(31859, 1)

Векторизируем признаки тренировчной и тестовой выборки отдельно друг от друга, учитывая стоп-слова.

In [36]:
corpus = features_train['lem_text'].values
corpus_test = features_test['lem_text'].values

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

vectorizer = TfidfVectorizer(stop_words=stopwords) 
features_train = vectorizer.fit_transform(corpus)
features_test = vectorizer.transform(corpus_test)

print(features_train.shape)
print(features_test.shape)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(127433, 132173)
(31859, 132173)


### Обучим 3 модели

In [37]:
%time
model = LogisticRegression(random_state=12345, class_weight='balanced')
parameters = [{'C': [8.7, 8.8, 8.6]}]

model_grs = GridSearchCV(model, param_grid=parameters, scoring='f1', n_jobs=-1, 
                   cv=3, verbose=False)
model_grs.fit(features_train, target_train)
print(model_grs.best_params_)
print(model_grs.best_score_)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
{'C': 8.6}
0.753450313643088


In [38]:
%%time
tree = DecisionTreeClassifier(random_state=12345)

param_grid_tree = [{'max_depth': range (1, 5)}]

grid_tree = GridSearchCV(tree, 
                         param_grid_tree, 
                         scoring = 'f1', 
                         cv=3, 
                         n_jobs = -1)

grid_tree.fit(features_train,target_train)
print(grid_tree.best_score_)
print(grid_tree.best_estimator_)

0.5061020743396659
DecisionTreeClassifier(max_depth=4, random_state=12345)
CPU times: user 1min 12s, sys: 518 ms, total: 1min 12s
Wall time: 1min 12s


In [39]:
#методом опорных векторов
%time
model_LSVC = LinearSVC()

parameters = [{'C': [0.5, 0.4, 0.3]}]


LSVC_model_gscv = GridSearchCV(model_LSVC, param_grid=parameters, scoring='f1', n_jobs=-1, 
                   cv=3, verbose=False)

LSVC_model_gscv.fit(features_train, target_train)

print(LSVC_model_gscv.best_params_)
print(LSVC_model_gscv.best_score_)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
{'C': 0.5}
0.7577258728041812


Наилучший результат показал метод опорных векторов. Проверим данную модель на тестовой выборке

In [40]:
preds_test = LSVC_model_gscv.predict(features_test)
f1_score(target_test, preds_test)

0.7802390998593529

## Выводы

- на первом этапе работы текст были лемматизированы и очищенны от неожиданных знаков, был учтен регистр сообщений (все символы приведены к нижнему)
- данные разделены на тестовую и тренировчную выборки, выделен целевой признак. Данные были векторизированы после разбиения на подвыборки в целях недопущения утечки 
- было обучено т=3 модели, две из которых на тренировочной выборке уже показали результат, превышающий целевой показатель F1-score равный 0,75
- наилучшая модель была проверена на тестовой выборке и показала результат 0,7802